<a target="_blank" href="https://colab.research.google.com/github/taobrienlbl/advanced_earth_science_data_analysis/blob/fall_2025_iub/content/lessons/04_digging_further_into_data_wrangling/04_estimating_building_height.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Exercise: Estimating Building Height with Pressure

This exercise aims to build on your knowledge of the big-four data science libraries to use Kestrel atmospheric measurement devices to estimate the height difference between the ground floor and sixth floor of the Geology building.

## Background

Air pressure monotonically decreases with height in the atmosphere.  This is simply due to the fact that *air pressure is a measure of the weight of everything above*.  So higher up in the atmosphere there is less atmosphere (weight) above, and pressure decreases.

One of the basic principles of fluid dynamics is that a *pressure gradient*--a change in pressure over some distance--exerts a net force on the fluid that, in the absence of other forces, would cause the fluid to move.  So why then does the atmosphere not flow away to space in response to this pressure gradient?

To a very good approximation, the atmosphere is in *hydrostatic balance*, meaning that the vertical pressure gradient is balanced by the force of gravity on the air; gravity effectively nullifies the vertical pressure gradient force.  This leads to one of the most essential equations in atmospheric science:

$$ \frac{\partial p}{\partial z} = -\rho g$$

where $p$ is the atmospheric pressure, $z$ is height away from the surface, $\rho$ is air density, and $g$ is the gravitational acceleration (we'll use $g \approx 9.806~\text{m s}^{-2}$ here).

If we combine this with the ideal gas law (written in a form that is convenient for atmospheric gasses, where $R_d \approx 287~\text{J kg}^{-1}\text{K}^{-1}$ represents a constant for air with the composition of Earth's atmosphere), we get a differential equation for pressure.

$$\text{Ideal gas law: }~p = \rho R_d T \to $$

$$ \frac{\partial p}{\partial z} = - \frac{p g}{R_d T} $$

We can approximate the derivatives in this equation using finite differences: $\partial p/\partial z \to \Delta p / \Delta z$, where $\Delta$ represents a (presumably small) difference between two pressure or height measurements.  We can then use this to solve for the change in height as a function of changes in pressure:

$$ \Delta z \approx - \frac{R_d T}{g} \frac{\Delta p}{p}$$

## Estimating height

Now imagine that we have a device that records pressure and temperature every second or so.  If you take that device and walk up a set of stairs, the recorded pressure will drop a bit for every second that you're ascending.  A pair of measurements taken at consecutive times can be used to estimate $\Delta p$, and the average of the two measurements can be used to estimate $p$ and $T$ over tha same time frame.  These values can be plugged in to the above formula to estimate the change in height between the two measurements.  The sum of all the changes in height then gives an estimate of the total change in height; we will have numerically integrated the differential equation that comes from combining hydrostatic balance and the ideal gas law.

### Instructions

1. form groups of two or three
1. acquaint yourself with the Kestrel measuring devices, and pair it to at least one of your phones using [Kestrel Link](https://kestrelinstruments.com/link-connectivity)
1. ensure that you can record measurements and export them to csv
1. take the kestrel device down to the ground floor of the Geology building; go to the lobby
1. start recording
1. either walk up the stairwell, or take the elevator, up to the 7th floor (you can only access this floor from the western stairwell)
1. end recording
1. export the data to a CSV file and add it to your course git folder (into a folder for lesson 04)
1. estimate the difference in height between the two floors using the method above (*hint*: you should use `pandas`, and you should be able to make a new column for the calculation of $\Delta p$ and $\Delta z$ for each measurement, and then use the `.sum()` method)

```{note}
If for some reason you need a data file to work with, here is the one used in the solution: [WEATHER_-_2414781_Sep_15_2023_2_56_14_PM.csv](https://raw.githubusercontent.com/taobrienlbl/advanced_earth_science_data_analysis/refs/heads/fall_2025_iub/content/lessons/04_digging_further_into_data_wrangling/WEATHER_-_2414781_Sep_15_2023_2_56_14_PM.csv)
```

In [33]:
# import modules.
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd


In [37]:
"""Read the data and sort them, get the pressure data as a column vector."""

# read the data from csv file named "kestrel_data.csv". The pressure data is in the range is D5:D60
data = pd.read_csv("kestrel_data.csv")

# get the column 3 as a numpy array
pressure = data.iloc[3:60, 3].to_numpy()

# convert the pressure data to float
pressure = pressure.astype(float)

# transform the pressure data to pascal, it was in mb
pressure = pressure * 100

# calculate the delta pressure, which is the difference between the cell and the cell next to it, use pandas diff function
df = pd.Series(pressure).diff().to_numpy()
# remove the first element which is nan
df = df[1:] 
delta_pressure = df
# see the data
print(delta_pressure)

# sum the delta pressure
sum_delta_pressure = np.sum(delta_pressure)
print("sum of delta pressure:", sum_delta_pressure)

# get the first column as a numpy array, which is the temperature data
temperature = data.iloc[3:60, 1].to_numpy()

# convert the temperature data to float
temperature = temperature.astype(float)

# convert the temperature data to kelvin, is was in celsius
temperature = temperature + 273.15

# see the data
print(temperature)

[-10.   0. -20.  20. -20.   0. -10. -10.   0.   0. -10.   0. -10. -10.
 -10.   0. -10.   0.  10. -10. -20.   0.   0. -10. -10.   0.   0. -10.
   0.   0. -10. -10. -20.   0.   0.   0. -10.   0.   0.  10. -30.  10.
   0. -20.  10. -40.   0.  10.  20. -20. -20.   0. -10.  10. -10.]
sum of delta pressure: -280.0
[294.75 294.75 294.85 294.65 294.55 294.35 294.35 294.35 294.25 294.15
 294.05 294.05 293.95 293.85 293.85 293.85 293.75 293.75 293.75 293.75
 293.75 293.75 293.75 293.85 293.85 293.85 293.85 293.85 293.95 293.95
 293.95 293.95 294.05 294.15 294.15 294.25 294.25 294.35 294.35 294.35
 294.25 294.35 294.35 294.35 294.35 294.35 294.45 294.45 294.45 294.45
 294.55 294.55 294.65 294.85 295.05 295.25]


In [39]:
"Convert the delta prssure to delta height"

# define functions
g = 9.806 # m/s^2 # gravitational acceleration
R_d = 287.05 # J/(kg*K) # specific gas constant for dry air

# calculate delta height using the formula, store the result in a column vector
delta_height = np.zeros(len(delta_pressure))
for i in range(len(delta_pressure)):
    delta_height[i] = - (R_d * temperature[i]) / g * (delta_pressure[i]) / pressure[i]

# sum the delta height to get the total height
total_height = np.sum(delta_height)
print("The estimated building height is: ", total_height, "m")

The estimated building height is:  24.391378319591862 m
